In [1]:
#!pip install networkx
#!pip install python-louvain
#!pip3 install networkit




In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from multiprocessing import Pool
import itertools
import community
import pandas as pd

import networkit as nk 


## Finding centrality of source twitter and target twitter account
 I couldn't install `NetworKit` here, it also causes my kernel to crash and restart on my local machine

In [2]:
df = pd.read_csv('social_network_data_2021_1227.csv')
df = df.dropna(subset=['source_twitter', 'target_twitter'])
print(df.shape)
df.head()

(258764, 16)


,source_twitter,target_twitter,source_location,target_location,follow_actions,interaction_times,interaction_type_counts,customer_type,tier,a_level,age_range,gender,orders,amount,first_authorized_at,first_payment_date
0,0120iesagashi,109btt,BTT,NaN,1,0,0,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN
1,0120iesagashi,16toyosu,BTT,NaN,1,0,0,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN
2,0120iesagashi,1manken,BTT,NaN,1,0,0,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN
3,0120iesagashi,2020wngn,BTT,NaN,1,1,1,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN
4,0120iesagashi,2022Btt,BTT,NaN,0,1,1,customer_registered_only,1-Bronze,a_level_0,not available,not available,NaN,NaN,2021-08-17 00:00:00.000,NaN


In [3]:
#df[['source_twitter', 'target_twitter']][:3000].to_csv('3k_accs.csv', index=False)

In [4]:
len(df['source_twitter'].unique()) + len(df['target_twitter'].unique()) 

178571

### Read with `NetworkX`

In [5]:
nxG = nx.from_pandas_edgelist(df, 
                            source='source_twitter',
                            target='target_twitter',
                            edge_attr=True,
                            create_using=nx.Graph())

print(nx.info(nxG))

Name: 
Type: Graph
Number of nodes: 178438
Number of edges: 258757
Average degree:   2.9002


### Keeping original node name
https://stackoverflow.com/questions/60285134/retrieving-original-node-names-in-networkit

In [6]:
# Get mapping from node ids in nxG to node ids in G
idmap = dict((id, u) for (id, u) in zip(nxG.nodes(), range(nxG.number_of_nodes())))

In [7]:
len(idmap)

178438

In [8]:
inverse_map = {v: k for k, v in idmap.items()}
print(type(inverse_map))
inverse_map_df = pd.DataFrame(inverse_map.items(), columns=['Index', 'Twitter'])
inverse_map_df

<class 'dict'>


,Index,Twitter
0,0,0120iesagashi
1,1,109btt
2,2,16toyosu
3,3,1manken
4,4,2020wngn
...,...,...
178433,178433,yuru_creation
178434,178434,yuzriha_h
178435,178435,yuzutouhu_ika
178436,178436,zaru_cr


In [9]:
inverse_map_df.iloc[315,:]

Index                 315
Twitter    0120sakura0120
Name: 315, dtype: object

### PageRank: Finding potential KOLs
#### A node is important if it is refered to by many other nodes

In [10]:
page_rank = nx.pagerank(nxG)

# sort by descending centrality
#page_rank = sorted(((c, v) for v, c in page_rank.items()), reverse=True)
page_rank =  pd.DataFrame(page_rank.items(), columns=['Twitter','PageRank'])
page_rank.head()

,Twitter,PageRank
0,0120iesagashi,0.000752
1,109btt,0.000003
2,16toyosu,0.000003
3,1manken,0.000003
4,2020wngn,0.000003


In [15]:
page_rank = page_rank.sort_values(by='PageRank', ascending=False)
page_rank = page_rank.rename(columns={'Twitter': 'Twitter_PR'})
page_rank

,Twitter_PR,PageRank
22271,KOICHIOSHIMA2,0.008360
141900,ru_irui_momo,0.008000
9954,Aoi66982994,0.007850
84400,iloveyou1984022,0.007798
96236,kitakubu918,0.006011
...,...,...
51274,WOMANEXPO,0.000002
51275,hekimasa,0.000002
51276,kominaao,0.000002
51277,momose_rei,0.000002


In [16]:
#page_rank.to_csv('page_rank.csv', index=False)